In [25]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score
from multiprocessing import Pool
import scipy as sc
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns =100
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import gensim
import pickle
from gensim.models import Word2Vec, FastText, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import sys
sys.path.insert(0, '..')
from Common import preprocessing,evaluation,ner,CosineClassifier as cos
classes_map = {'DOC':0, 'ENTER':1, 'ORG':2, 'PRIV':3, 'RANG':4, 'HOST':5}

In [26]:
df = pd.read_csv('..//Data//data.txt', delimiter=';', engine='python',encoding='utf8')

In [27]:
questions = np.array(df.question)
questions = preprocessing.preprocess_list(questions)

In [28]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
X = vectorizer.fit_transform(questions)

In [29]:
classes = np.array(df['class'])
y = list(map(lambda x: classes_map[x],classes))

In [33]:
log_reg = OneVsRestClassifier(LogisticRegression(random_state=0,C=10,solver='lbfgs'))

In [39]:
print(evaluation.get_classification_report(log_reg,embeddings,y))

             precision    recall  f1-score   support

        DOC       0.76      0.77      0.76        65
      ENTER       0.58      0.75      0.65        68
        ORG       0.75      0.73      0.74       151
       PRIV       0.60      0.21      0.32        14
       RANG       0.75      0.74      0.75        98
       HOST       0.98      0.85      0.91        47

avg / total       0.75      0.74      0.74       443



In [9]:
print(evaluation.get_CV_scores(log_reg,X,y,'precision_micro').mean())
print(evaluation.get_CV_scores(log_reg,X,y,'recall_micro').mean())
print(evaluation.get_CV_scores(log_reg,X,y,'f1_micro').mean())

0.7548604560838927
0.7548604560838927
0.7548604560838927


In [21]:
model = Word2Vec([q.split() for q in questions], size=300, window=10,)
model.train([q.split() for q in questions],epochs=500,total_examples=model.corpus_count)

(3916286, 6809500)

In [ ]:
model = FastText([q.split() for q in questions], size=300, window=10,)
model.train([q.split() for q in questions],epochs=200,total_examples=model.corpus_count)

Exception in thread Thread-827:
Traceback (most recent call last):
  File "c:\users\aleksandr.perevalov\appdata\local\continuum\miniconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\aleksandr.perevalov\appdata\local\continuum\miniconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\aleksandr.perevalov\appdata\local\continuum\miniconda3\lib\site-packages\gensim\models\base_any2vec.py", line 211, in _worker_loop
    tally, raw_tally = self._do_train_job(data_iterable, job_parameters, thread_private_mem)
  File "c:\users\aleksandr.perevalov\appdata\local\continuum\miniconda3\lib\site-packages\gensim\models\fasttext.py", line 595, in _do_train_job
    tally += train_batch_cbow(self, sentences, alpha, work, neu1)
  File "gensim/models/fasttext_inner.pyx", line 431, in gensim.models.fasttext_inner.train_batch_cbow
TypeError: 'NoneType' object is not subscriptable

Exception in thread Thread-826:
Traceb

In [10]:
doc2vecs = []

for q in questions:
    if len(q) < 1:
        doc2vecs.append(np.zeros(300))
    else:
        doc2vecs.append(doc2vec.infer_vector(q))

In [22]:
word2vec = []

for q in questions:
    vec = [model.wv[w] for w in q.split() if w in model.wv]
    if len(vec) < 1:
        word2vec.append(np.zeros(300))
    else:
        word2vec.append(np.array(np.mean(vec,axis=0)))

In [10]:
enthropy = []
for q in questions:
    question = q.split()
    q_dic = Counter(question)
    q_len = len(question)
    vector = []
    cnt=0
    for w in vectorizer.get_feature_names():
            if w in question:
                vector.append(q_dic[w]/q_len*np.log2(q_dic[w]/q_len))
            else:
                vector.append(-0.00001)
            cnt=cnt+1
    enthropy.append(vector.copy())

In [11]:
svd = TruncatedSVD(n_components=200,algorithm='arpack')
matr = svd.fit_transform(np.array(enthropy).T)

In [12]:
matr = Normalizer().fit_transform(matr)

In [13]:
enth = {}
for i in list(zip(vectorizer.get_feature_names(),matr)):
    enth[i[0]]=[i[1]]

In [14]:
embeddings = []

for q in questions:
    vec = [enth[w][0] for w in q.split() if w in list(enth.keys())]
    if len(vec) < 1:
        embeddings.append(np.zeros(200))
    else:
        embeddings.append(np.array(np.mean(vec,axis=0)))

In [36]:
count_model = CountVectorizer(min_df=3,ngram_range=(1,1))
X_ = count_model.fit_transform(questions)
Xc = (X_.T * X_)
Xc.setdiag(0)
co_occur = Normalizer().fit_transform(Xc.todense())
print(Xc.todense().shape)

(748, 748)


In [8]:
pd.DataFrame(word2vec).to_csv("word2vec.tsv", sep='\t',index=False,header=None)

In [29]:
pairs = []
for i in range(matr.shape[0]):
    for j in range(matr.shape[0]):
        if i!=j and vectorizer.get_feature_names()[i] in model.wv and vectorizer.get_feature_names()[j] in model.wv:
            if cosine_similarity([model.wv[vectorizer.get_feature_names()[i]]],[model.wv[vectorizer.get_feature_names()[j]]]) > 0.5:
                if str(i)+' '+str(j) not in pairs and str(j)+' '+str(i) not in pairs:
                    pairs.append(str(i)+' '+str(j))
                    mean = np.mean(np.mean([matr[i],matr[j]],axis=0))
                    matr[i] = mean
                    matr[j] = mean
                    print(vectorizer.get_feature_names()[i],vectorizer.get_feature_names()[j])

абитуриент зелёный
адрес корочка
адрес отправлять
адрес письмо
адрес посылать
адрес точный
адрес эл
английский иностранный
английский интересовать
английский язык
анкета заполнить
анкета ксерокопия
анкета серия
анкета страница
армия действительный
армия летом
армия осень
армия отсрочка
армия пойти
армия призыв
армия сын
асу заочко
асу нгд
асу расписание
асу ускоренный
аудитория заочник
аудитория предоставлять
аудитория присутствовать
аудитория сколька
аэрокосмический одновременно
база вместо
база проживать
бакалавр кафедра
безопасность горный
безопасность информационный
безопасность система
билет собрание
билет студенческий
большой маленький
будущее комплекс
будущее олимпиада
бф пнипа
бюджетник социальный
бюджетник стипендия
бюджетник студенческий
вакантный равный
взять отпуск
вид почта
вид эл
вид электронный
видеть выложить
внутренний действительный
военкомат новый
военный кафедра
военный служба
восстановиться гнф
восстановиться осень
восстановиться очно
восстановиться перейти
временн

KeyboardInterrupt: 